### recommindation_for_newbie
Базовая рекомендация курсов на основании оценок пользователей и количества отзывов по курсу - холодный старт
                #для нового пользователя
                #по запросу "выучить что-нибудь новое"
Результат: список названий топовых курсов по следующей метрике: средняя оценка курса, умноженная на нормированное количество рейтингов
    
''' проверяем удовлетворённость юзера и повторяем вывод рекомендаций, пока юзер не ответит 'yes' '''

#Источник данных: Oбъединённый датафрейм описания англоязычных курсов с отзывами пользователей 'joined_users_reviews.csv'

In [1]:
import pandas as pd
import numpy as np

In [2]:
joined_users_reviews = pd.read_csv('joined_users_reviews.csv')
joined_users_reviews.head(2)

,Unnamed: 0,course_id,reviewer_name,rating,review_text,title,topics,about,instructors,language,metrics,product_glance,average_score,ratings_count,reviews_count,skills,syllabus,recommendations,url
0,0,2-speed-it,Ravish,5,Very relevant and useful course designed for CIOs,Two Speed IT: How Companies Can Surf the Digit...,Business Business Essentials,"Transform or disappear, the Darwinism of IT: I...",Antoine Gourévitch Vanessa Lyon Eric Baudson,English,16728 already enrolled! 5149 recent views,100% online Flexible deadlines Approx. 21 hour...,4.4,33,33,NaN,Introduction IT and the CIO in the Digital Wor...,fundamentals-of-management entrepreneurial-thi...,https://www.coursera.org/learn/2-speed-it
1,1,2-speed-it,Etienne R,2,This course does not say anything about digiti...,Two Speed IT: How Companies Can Surf the Digit...,Business Business Essentials,"Transform or disappear, the Darwinism of IT: I...",Antoine Gourévitch Vanessa Lyon Eric Baudson,English,16728 already enrolled! 5149 recent views,100% online Flexible deadlines Approx. 21 hour...,4.4,33,33,NaN,Introduction IT and the CIO in the Digital Wor...,fundamentals-of-management entrepreneurial-thi...,https://www.coursera.org/learn/2-speed-it


In [3]:
reviews_eng_courses = pd.read_csv('reviews_eng_courses_1.csv')
reviews_eng_courses.head(3)

,Unnamed: 0,course_id,reviewer_name,rating,review_text
0,0,2-speed-it,Ravish,5,Very relevant and useful course designed for CIOs
1,1,2-speed-it,Etienne R,2,This course does not say anything about digiti...
2,2,2-speed-it,Viswas P,4,Videos that are presented in French could've b...


In [4]:
#Базовая рекомендация курсов на основании оценок пользователей - холодный старт
# посмотрим на количество рейтингов по каждому курсу 
course_id_num_ratings = {}
for i, group in reviews_eng_courses.groupby('course_id'):course_id_num_ratings[i] = group.reviewer_name.unique().shape[0]

# возьмем топ курсов по оценкам
top_courses = reviews_eng_courses.groupby('course_id')[['rating']].mean().sort_values('rating', ascending=False)
# из них возьмем курсы с наивысшей средней оценкой в 5.0
courses_with_highest_rating = top_courses.iloc[np.where(top_courses.rating == 5.0)].index
#в принципе эту выборку уже можно использовать для самой простой базовой рекомендации новому пользователю
courses_with_highest_rating

# выведем топ курсов со средней оценкой в 5.0 по количеству отзывов
sorted([(course_id_num_ratings[f], f) for f in courses_with_highest_rating], key=lambda x: x[0], reverse=True)[:10]

# средний рейтинг на каждый курс
course_id_mean_rating = {}

for i, group in reviews_eng_courses.groupby('course_id'):course_id_mean_rating[i] = group.rating.mean()
    
# достанем простые статистики по количеству рейтингов
min_num_ratings = np.min([course_id_num_ratings[f] for f in course_id_num_ratings.keys()])
max_num_ratings = np.max([course_id_num_ratings[f] for f in course_id_num_ratings.keys()])
mean_num_ratings = np.mean([course_id_num_ratings[f] for f in course_id_num_ratings.keys()])
median_num_ratings = np.median([course_id_num_ratings[f] for f in course_id_num_ratings.keys()])

In [5]:
#Рекомендация курсов на основании оценок и количества отзывов по курсу
#для нового пользователя
#по запросу "выучить что-нибудь новое"

# Oбъединённый датафрейм описания англоязычных курсов с отзывами пользователей 
joined_users_reviews = pd.read_csv('joined_users_reviews.csv')
#избавимся от лишних столбцов
joined_users_reviews=joined_users_reviews.drop(['Unnamed: 0'], axis=1)
joined_users_reviews['skills'] = joined_users_reviews.skills.fillna(value = '')
#подготовленный датасет
data = joined_users_reviews

# отсортируем курсы по следующей метрике: средняя оценка курса, умноженная на нормированное количество рейтингов
course_new_mark = []
for f in course_id_num_ratings.keys(): course_new_mark.append((f, course_id_mean_rating[f] * (course_id_num_ratings[f] - mean_num_ratings) / (max_num_ratings - min_num_ratings)))
        
# сортируем курсы по средним оценкам и количеству рейтингов
top_ = dict(sorted(course_new_mark, key=lambda x: x[1], reverse=True))
dict_keys = list(top_.keys()) # список course_id - результат отбора и сортировки - решение задачи 1

In [6]:
dict_keys[0:5]

['matlab',
 'game-development',
 'project-management-basics',
 'science-of-meditation',
 'algorithmic-toolbox']

In [8]:
dict_keys

['matlab',
 'game-development',
 'project-management-basics',
 'science-of-meditation',
 'algorithmic-toolbox',
 'mindshift',
 'data-cleaning',
 'language-theories',
 'data-science-course',
 'learn-to-program',
 'philosophy',
 'uva-darden-design-thinking-innovation',
 'ml-regression',
 'positive-psychology-visionary-science',
 'professional-emails-english',
 'object-oriented-java',
 'careerdevelopment',
 'introduction-psych',
 'website-coding',
 'search-engine-optimization',
 'introcss',
 'python-programming-introduction',
 'gis',
 'wharton-quantitative-modeling',
 'arduino-platform',
 'positive-psychology',
 'interactive-python-1',
 'probability-intro',
 'dog-emotion-and-cognition',
 'introduction-to-software-product-management',
 'excel-data-analysis',
 'exploratory-data-analysis',
 'what-is-social',
 'sustainable-development',
 'decision-making',
 'financial-markets',
 'psychological-first-aid',
 'craft-of-plot',
 'javascript',
 'python-data-visualization',
 'basic-statistics',
 'sp

In [9]:
df_by_course_title = pd.DataFrame()
m=0
n=5 # задаём количество курсов для вывода
    
# выводим топ  курсов по средним оценкам и количеству рейтингов с названиями и ссылками на курсы
print('Топ курсов по средним оценкам и количеству рейтингов: ')
for k in dict_keys[m:n]: 
    filter_course_id = data['course_id'] == k
    df_by_course_id = data.loc[filter_course_id][0:1]
    df_by_course_title = df_by_course_title.append(df_by_course_id[['title', 'url']])
    
for i in range(n): 
    a = df_by_course_title.iloc[i].values
    print(a[0], ' ', a[1])  


Топ курсов по средним оценкам и количеству рейтингов: 
Introduction to Programming with MATLAB   https://www.coursera.org/learn/matlab
Introduction to Game Development   https://www.coursera.org/learn/game-development
Project Management: The Basics for Success   https://www.coursera.org/learn/project-management-basics
Buddhism and Modern Psychology    https://www.coursera.org/learn/science-of-meditation
Algorithmic Toolbox   https://www.coursera.org/learn/algorithmic-toolbox


In [13]:
'''Are you satisfied? Print: Yes / No'''
m=0
n=5# задаём количество курсов для вывода
yes_no_answer = 'No'
df_by_course_title = pd.DataFrame()

    
while yes_no_answer == 'No' or yes_no_answer == 'no':
        # внутри цикла повторяем смещение выбранного слайса списка отсортированных курсов для вывода результата рекомендации, пока ответ не = 'да'
        # или пока не дойдём до конца списка отсортированных курсов
    df_by_course_title = pd.DataFrame()
    
    # выводим топ n курсов по средним оценкам и количеству рейтингов с названиями и ссылками на курсы
    print('I highly recommend: ')
    for k in dict_keys[m:n]:
        filter_course_id = data['course_id'] == k
        df_by_course_id = data.loc[filter_course_id][0:1]
        df_by_course_title = df_by_course_title.append(df_by_course_id[['title', 'url']])

    for i in range(0,n-m):
        a = df_by_course_title.iloc[i].values
        print(a[0], ' ', a[1])  
    
    print('---')
    print('Are you satisfied? Print: Yes / No')
    yes_no_answer = input()

    m+=5
    n+=5 # задаём смещение выбранного слайса списка отсортированных курсов
    

I highly recommend: 
Introduction to Programming with MATLAB   https://www.coursera.org/learn/matlab
Introduction to Game Development   https://www.coursera.org/learn/game-development
Project Management: The Basics for Success   https://www.coursera.org/learn/project-management-basics
Buddhism and Modern Psychology    https://www.coursera.org/learn/science-of-meditation
Algorithmic Toolbox   https://www.coursera.org/learn/algorithmic-toolbox
---
Are you satisfied? Print: Yes / No
no
I highly recommend: 
Mindshift: Break Through Obstacles to Learning and Discover Your Hidden Potential   https://www.coursera.org/learn/mindshift
Getting and Cleaning Data   https://www.coursera.org/learn/data-cleaning
Teach English Now! Theories of Second Language Acquisition   https://www.coursera.org/learn/language-theories
A Crash Course in Data Science   https://www.coursera.org/learn/data-science-course
Learn to Program: The Fundamentals   https://www.coursera.org/learn/learn-to-program
---
Are you sa